In [1]:
import pandas as pd
import sys
sys.path.append("../../Muestreador")
from muestreador import MuestreoINE
m = MuestreoINE()
from collections import defaultdict

In [2]:
df = pd.read_csv("../Datos/radon.csv")
df.head(6)

,countyname,countynum,sampsize,popsize,radon
0,Aitkin,1,4,5400,1.0
1,Aitkin,1,4,5400,2.2
2,Aitkin,1,4,5400,2.2
3,Aitkin,1,4,5400,2.9
4,Anoka,2,57,71900,2.4
5,Anoka,2,57,71900,0.5


In [3]:
df["sampleweight"] = df["sampsize"] / df["popsize"]
df

,countyname,countynum,sampsize,popsize,radon,sampleweight
0,Aitkin,1,4,5400,1.0,0.000741
1,Aitkin,1,4,5400,2.2,0.000741
2,Aitkin,1,4,5400,2.2,0.000741
3,Aitkin,1,4,5400,2.9,0.000741
4,Anoka,2,57,71900,2.4,0.000793
...,...,...,...,...,...,...
998,Wright,86,14,21600,3.1,0.000648
999,Wright,86,14,21600,8.7,0.000648
1000,Yellow Medicine,87,3,4600,3.7,0.000652
1001,Yellow Medicine,87,3,4600,2.9,0.000652


In [4]:
m.media_estratificado("sampleweight", "radon", df)

3.6234473894170898

In [5]:
tamaño_estratos = df.groupby(["countynum"])["popsize"].first().tolist()
tamaño_muestras = df.groupby(["countynum"])["sampsize"].first().tolist()
m.media_IC_estratificado_z(0.95, tamaño_estratos, tamaño_muestras, "sampleweight", df, "countynum", "radon")

(2.0629538112373424, 5.183940967596837)

In [6]:
# Obtenemos las frecuencias totales de cada 'countyname' y aquellas con 'radon' >= 4
frecuencias_estratos = df["countyname"].value_counts()
frecuencias_incidencia = df[df["radon"] >= 4]["countyname"].value_counts()

# Inicializamos los diccionarios con las frecuencias obtenidas
estratos_con_frecuencia = frecuencias_incidencia.to_dict(defaultdict(int))
ocurrencia_estratos = frecuencias_estratos.to_dict()

# Calculamos las proporciones directamente usando comprensión de diccionarios
proporciones_dic = {estrato: estratos_con_frecuencia.get(estrato, 0) / ocurrencia_estratos[estrato] 
               for estrato in ocurrencia_estratos.keys()}


In [7]:
proporciones = list(proporciones_dic.values())
m.proporcion_IC_estratificado(0.95, tamaño_estratos, tamaño_muestras, proporciones)

(0.5939202189556942, 0.627814557868448)